In [75]:
import pandas as pd
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER
import re
data = pd.read_csv(r'C:\Users\Jasonhsu\Desktop\esun\0611\content_df_0612.csv')

In [76]:
data.head()

,news_id,url,context,name,raw_content,status
0,1,https://news.cnyes.com/news/id/4352432,0理財基金量化交易追求絕對報酬有效對抗牛熊市鉅亨網記者鄭心芸2019/07/05 22:35...,[],熱搜：熱門行情最近搜尋基金老司機近年來投資市場波動越來越明顯，追求低波動、絕對報酬的量化交易...,ok
1,2,https://udn.com/news/story/120775/4112519,10月13日晚間發生Uber Eats黃姓外送人員職災死亡案件 ### 省略內文 ### 北...,[],10月13日晚間發生Uber Eats黃姓外送人員職災死亡案件，北市府勞動局認定業者未依職業...,ok
2,3,https://www.mirrormedia.mg/story/20191008soc011/,2019.10.08 01:53【法拍有詭4】飯店遭管委會斷水斷電員工怒吼：生計何去何從？文...,[],\n 全部類別\n \n 全部類別\n 【法拍有詭1】3個月...,ok
3,4,https://www.chinatimes.com/realtimenews/201909...,58歲林姓男子昨凌晨與朋友聚餐結束後 ### 省略內文 ### 而地點就距離林家僅30公尺。...,[],NaN,404NotFound
4,5,http://domestic.judicial.gov.tw/abbs/wkw/WHD9H...,例稿名稱：臺灣屏東地方法院公示催告公告發文日期：中華民國108年9月20日發文字號：屏院進家...,[],例稿名稱：臺灣屏東地方法院公示催告公告\n發文日期：中華民國108年9月20日\n發文字號：...,ok


In [77]:
data=data[data["status"]=="ok"].reset_index().drop(["url","context","status","index"],axis =1)

In [78]:
data['aml_label'] = data['name'].apply(lambda x: 0 if x == '[]' else 1)

In [79]:
#data['raw_content']=data['raw_content'].apply(lambda x: max(x.split('    '), key=len))

In [80]:
#data.to_csv('xxxxx.csv',index=False)

In [81]:
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime
import os
print("tensorflow version : ", tf.__version__)
print("tensorflow_hub version : ", hub.__version__)

tensorflow version :  1.15.0
tensorflow_hub version :  0.8.0


In [82]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization
from bert import modeling

In [83]:
OUTPUT_DIR = r'C:\Users\Jasonhsu\Desktop\esun\bert_classfication2'

In [84]:
train, val =  train_test_split(data, test_size = 0.5, random_state = 2020)
train.to_csv('train.csv',index=False)
val.to_csv('val.csv',index=False)

In [85]:
print(train['aml_label'].value_counts())
print(val['aml_label'].value_counts())

0    2249
1     178
Name: aml_label, dtype: int64
0    2275
1     153
Name: aml_label, dtype: int64


In [86]:
DATA_COLUMN = 'raw_content'
LABEL_COLUMN = 'aml_label'
label_list = [0, 1]

In [87]:
train_InputExamples = train.apply(lambda x: bert.run_classifier.InputExample(guid= None,
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

val_InputExamples = val.apply(lambda x: bert.run_classifier.InputExample(guid= None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

In [88]:
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_chinese_L-12_H-768_A-12/1"

def create_tokenizer_from_hub_module():

    with tf.Graph().as_default():
        bert_module = hub.Module(BERT_MODEL_HUB)
        tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
        with tf.Session() as sess:
            vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],tokenization_info["do_lower_case"]])
      
    return bert.tokenization.FullTokenizer(vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

In [89]:
MAX_SEQ_LENGTH = 256
#轉換token為bert輸入
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

val_features = bert.run_classifier.convert_examples_to_features(val_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

In [90]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
  
    bert_module = hub.Module(BERT_MODEL_HUB,trainable=True)
    bert_inputs = dict(input_ids=input_ids,input_mask=input_mask,segment_ids=segment_ids)
    bert_outputs = bert_module(inputs=bert_inputs,signature="tokens",as_dict=True)

    output_layer = bert_outputs["pooled_output"]

    hidden_size = output_layer.shape[-1].value

    output_weights = tf.get_variable("output_weights", [num_labels, hidden_size],initializer=tf.truncated_normal_initializer(stddev=0.02))

    output_bias = tf.get_variable("output_bias", [num_labels], initializer=tf.zeros_initializer())

    with tf.variable_scope("loss"):

        output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

        logits = tf.matmul(output_layer, output_weights, transpose_b=True)
        logits = tf.nn.bias_add(logits, output_bias)
        log_probs = tf.nn.log_softmax(logits, axis=-1)

        one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

        predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))

    if is_predicting:
        return (predicted_labels, log_probs)

    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)

In [91]:
def model_fn_builder(num_labels, learning_rate, num_train_steps,num_warmup_steps):
    def model_fn(features, labels, mode, params):

        input_ids = features["input_ids"]
        input_mask = features["input_mask"]
        segment_ids = features["segment_ids"]
        label_ids = features["label_ids"]

        is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
        if not is_predicting:
            (loss, predicted_labels, log_probs) = create_model(is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

            train_op = bert.optimization.create_optimizer(loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

            def metric_fn(label_ids, predicted_labels):
                accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
                true_pos = tf.metrics.true_positives(label_ids,predicted_labels)
                true_neg = tf.metrics.true_negatives(label_ids,predicted_labels)   
                false_pos = tf.metrics.false_positives(label_ids,predicted_labels)  
                false_neg = tf.metrics.false_negatives(label_ids,predicted_labels)
        
                return {
                    "eval_accuracy": accuracy,
                    "true_positives": true_pos,
                    "true_negatives": true_neg,
                    "false_positives": false_pos,
                    "false_negatives": false_neg
                    }

            eval_metrics = metric_fn(label_ids, predicted_labels)

            if mode == tf.estimator.ModeKeys.TRAIN:
                return tf.estimator.EstimatorSpec(mode=mode,
                  loss=loss,
                  train_op=train_op)
            else:
                return tf.estimator.EstimatorSpec(mode=mode,loss=loss,eval_metric_ops=eval_metrics)
        else:
            (predicted_labels, log_probs) = create_model(is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

            predictions = {
            'probabilities': log_probs,
            'labels': predicted_labels
            }
            return tf.estimator.EstimatorSpec(mode, predictions=predictions)

    return model_fn

In [92]:
BATCH_SIZE = 8
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 5.0
# Warmup is a period of time where the learning rate is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 300
SAVE_SUMMARY_STEPS = 100

# Compute train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

# Specify output directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

# Specify output directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [93]:
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})

In [94]:
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

val_input_fn = run_classifier.input_fn_builder(
    features=val_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

In [95]:
print(f'Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

Beginning Training!
Training took time  0:00:00.006755


In [96]:
estimator.evaluate(input_fn=val_input_fn, steps=None)

C:\Users\Jasonhsu\anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


{'eval_accuracy': 0.99011534,
 'false_negatives': 20.0,
 'false_positives': 4.0,
 'loss': 0.061410517,
 'true_negatives': 2271.0,
 'true_positives': 133.0,
 'global_step': 1516}

In [97]:
def getPrediction(in_sentences):
    labels = ['non_aml', 'aml']
    input_examples = [run_classifier.InputExample(guid="", text_a = x, text_b = None, label = 0) for x in in_sentences] 
    input_features = run_classifier.convert_examples_to_features(input_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
    predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
    predictions = estimator.predict(predict_input_fn)
    return [(sentence, labels[prediction['labels']]) for sentence, prediction in zip(in_sentences, predictions)]
#   return [(sentence, prediction['probabilities'],prediction['labels'], labels[prediction['labels']]) for sentence, prediction in zip(in_sentences, predictions)]

In [166]:
def split_sentence(sentence):
    split_data=[x for x in re.split(',|。|，',sentence) if x]
    sentence_3=[]
    seq=''
    for i in range(0,len(split_data)-1,3):
        try:
            sentence_3.append(seq.join((split_data[i],',',split_data[i+1],',',split_data[i+2])))
        except:
            try:
                sentence_3.append(seq.join((split_data[i],',',split_data[i+1])))
            except:
                sentence_3.append(split_data[i])
    return sentence_3

In [167]:
data['split_sentence'] = data['raw_content'].apply(lambda x: split_sentence(x))

In [129]:
ws = WS("./data")
pos = POS("./data")
ner = NER("./data")

In [ ]:
def create_name_list(into_list):
    name_list=[]
    for i in into_list:
        if i[2]=='PERSON':
            name_list.append(i[3])
    return name_list

In [161]:
entity_sentence_list = ner(ws(data['split_sentence'][0]), pos(ws(data['split_sentence'][0])))

In [162]:
list(map(lambda x:create_name_list(x),entity_sentence_list))

[[],
 [],
 [],
 [],
 [],
 [],
 ['張堯勇'],
 [],
 [],
 ['詹姆斯 · 西蒙斯 ', 'James Simons'],
 ['索羅斯', '巴菲特'],
 [],
 ['張堯勇'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['張堯勇'],
 [],
 [],
 [],
 ['張堯勇'],
 [],
 [],
 [],
 [],
 ['8 兆', '張堯勇'],
 []]

In [ ]:
data['get_name']=data['split_sentence'].apply()

In [114]:
def print_word_pos_sentence(word_sentence, pos_sentence):
    assert len(word_sentence) == len(pos_sentence)
    for word, pos in zip(word_sentence, pos_sentence):
        print(f"{word}({pos})", end="\u3000")
    print()
    return
    
for i, sentence in enumerate(sentence_list):
    print()
    print(f"'{sentence}'")
    print_word_pos_sentence(word_sentence_list[i],  pos_sentence_list[i])
    for entity in sorted(entity_sentence_list[i]):
        print(entity)


'10月13日晚間發生Uber Eats黃姓外送人員職災死亡案件，北市府勞動局認定業者未依職業安全衛生法第37條第2項第1款規定於8小時內向勞動檢查機構通報，開罰Uber Eats 30萬元。

台北市勞動局表示，經勞動檢查發現，Uber Eats在台灣有稅籍登記（境外電商登記），但無公司登記，透過委任或服務合作備忘錄方式，分別與香港商比錫茲有限公司、易得汽車貨運股份有限公司簽訂合作契約，易得公司再與外送員以電腦平台方式簽訂合約，而合約內容卻由Uber Eats提供，且外送員之工資亦由Uber Eats直接支付，故勞動局認定Uber Eats為外送員之實質雇主。

北市勞動局職安科科長康水順表示，職安衛部份已完成檢查，經訪談筆錄及取得的文件認定該名罹災勞工與Uber Eats具有僱傭關係，故依職業安全衛生法第37條第2項第1款，雇主應於8小時內通報，但至今無收到任何通報記錄；另外送員於發生職災死亡後，Uber Eats知悉事故的發生卻未通報，屬明知法令卻仍違法。

北市勞動局長賴香伶表示，由於平台業者標榜食物外送之快捷性，卻罔顧外送員行車安全，更以送達時間計算外送員工作獎金，致外送員為達平台業者規定，執行外送業務時於道路上高速行駛及穿梭於車陣，徒增行車事故風險。勞動局已事先向業者宣導法令，考量Uber Eats違法行為嚴重影響公共秩序，受責難程度高，依行政罰法、職業安全衛生法裁處罰鍰30萬元。

勞動局職安科表示，今日先針對職災通報部分裁處作成決議，後續業者違反勞基法的部分，將根據違法部分另行開罰。除針Uber Eats黃姓外送人員職災死亡卻未依法通報案件開罰業者30萬元，北市政府今日下午也跨局處召開「外送人員保障專案會議」。圖／台北市勞動局提供
                    外送員車禍又一樁！Uber Eats林姓外送員與柯姓女駕駛發生碰撞，林男身體擦傷，送醫後無礙。近日外送平台崛起，路上可見...                  
                    新北市府交通局今天表示，將規劃機車卸貨停車區域，並開放外送員臨時停靠家長接送區，公有停車場也提供機車免費停車時段，方便機...                  
                    近來外送產業爭議不斷，台北市政府警察局今邀集交通局及Uber eats、Foo

隨機測試

In [86]:
data['raw_content'].str.replace('\(.*?\)', '')[0]

'熱搜：熱門行情最近搜尋基金老司機近年來投資市場波動越來越明顯，追求低波動、絕對報酬的量化交易備受注目。專家表示，採用量化交易策略投資台股，不管是處於多頭或是空頭市場，績效及波動度均可領跑大盤，甚至比國內投資台股的股票型基金及 ETF 的波動率還低，表現也更為穩定。大數據時代來臨，風行歐美 50 年的量化交易儼然成為顯學，台灣亦開始重視此一趨勢發展，也因此，中華機率統計學會及台北科技大學管理學院攜手主辦，並由元大期貨、摩根亞太量化交易等公司擔任協辦單位，今  日舉辦「時間序列與量化交易研討會」，就目前熱門的量化交易、智能投資等相關議題進行研討。越來越多的基金公司重視量化交易，全球規模較大的避險基金多採行量化交易，包括橋水基金 、AQR 資產管理公司、曼氏集團 、文藝復興科技  等全球知名避險基金。摩根亞太集團董事長張堯勇指出，避險基金規模約為 5 兆美元，採取量化交易的基金規模約 1 兆美元，比重佔了 20%，代表量化交易的操作績效好，才會有那麼高的比重。量化交易的操作績效不亞於價值投資及技術投資，被譽為數學天才、最賺錢的基金經理人，文藝復興對沖基金創始人詹姆斯 · 西蒙斯  所管理的大獎章  基金，便是典型的量化交易，績效表現優異，不僅勝過索羅斯的量子基金，也打敗了股神巴菲特的價值投資。近年來台灣也逐漸重視量化交易，摩根亞太量化交易公司今年開始將量化交易引進台股投資，是國內首家推出量化交易策略的公司，初期對象鎖定法人機構，進行私募投資。張堯勇表示，數學不只是一門學科，更是一項扭轉乾坤、轉敗為勝的競爭利器，量化交易就是將數學運用在股市投資，透過複雜、精密的推理計算，打敗股市。他指出，目前全球利率水平 50 位於年來新低，美國十年期公債殖利率只有 2%，日本及一些歐洲國家甚至是負利率。歷史極低利率帶導致無風險及低風險工具的投資報酬率太低（例如：銀行存款、政府公債及投資等級公司債），無法對抗通貨膨脹及支付負債，所以資產配置必須增加較高風險的投資，如股票、高收益公司債或新興市場債券，但是這些投資工具波動很大，而且很容易產生虧損！他表示，長期來說，股市絕對是好的投資，報酬率也不差，但是如何選股是一門學問。目前全球 60 個主要股市交易所，總市值 70 兆美元，而台灣上市上櫃市值 35 兆台幣，面對全球及台灣最大的股市金礦，投資人應該善用「量化交易」這個超級挖礦機，以程式選股

In [24]:
tests = getPrediction([
    '繼員林站1名旅客落軌遭自強號列車撞上後，台鐵剛剛又發生一起事故。',
    '留美蔡姓少校涉嫌提供郵局帳戶等資料，給予詐欺集團行騙近30萬元，被屏東地院依詐欺罪，判處有期徒刑5月，案經上訴高雄高分院後，宣告緩刑3年定讞，緩刑期間交付保護管束，並賠償被害人共15萬元。'
    ])

INFO:tensorflow:Writing example 0 of 2


INFO:tensorflow:Writing example 0 of 2


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:Restoring parameters from C:\Users\Jasonhsu\Desktop\esun\bert_classfication2\model.ckpt-1495


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


In [36]:
getPrediction(['專案小組認定國安局少校吳宗憲、張恒嘉與華航空品處前副總邱彰信、協理于堯、代理組長黃川禎等人涉犯貪汙治罪條例，',
               '另買菸大戶上士駕駛劉尊彰、民間人士李宗原等人涉違反稅捐稽徵法，依法起訴。其餘單純團購人員則另簽分偵辦。',
               '7月22日，總統蔡英文結束12天「自由民主永續之旅」返台，調查局以行政檢查方式，',
              '攔查國安局官員隨團走私9800條菸，金額高達645萬元，'
              '創桃園機場夾帶走私菸最大數量紀錄。'])


INFO:tensorflow:Writing example 0 of 4


INFO:tensorflow:Writing example 0 of 4


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


[('專案小組認定國安局少校吳宗憲、張恒嘉與華航空品處前副總邱彰信、協理于堯、代理組長黃川禎等人涉犯貪汙治罪條例，',
  array([-8.4189816e+00, -2.2063205e-04], dtype=float32),
  1,
  'aml'),
 ('另買菸大戶上士駕駛劉尊彰、民間人士李宗原等人涉違反稅捐稽徵法，依法起訴。其餘單純團購人員則另簽分偵辦。',
  array([-6.3539352e+00, -1.7414418e-03], dtype=float32),
  1,
  'aml'),
 ('7月22日，總統蔡英文結束12天「自由民主永續之旅」返台，調查局以行政檢查方式，',
  array([-7.700623e-05, -9.471053e+00], dtype=float32),
  0,
  'non_aml'),
 ('攔查國安局官員隨團走私9800條菸，金額高達645萬元，創桃園機場夾帶走私菸最大數量紀錄。',
  array([-9.255009e+00, -9.560128e-05], dtype=float32),
  1,
  'aml')]

In [25]:
tests

[('繼員林站1名旅客落軌遭自強號列車撞上後，台鐵剛剛又發生一起事故。',
  array([-6.413254e-05, -9.654260e+00], dtype=float32),
  0,
  'non_aml'),
 ('留美蔡姓少校涉嫌提供郵局帳戶等資料，給予詐欺集團行騙近30萬元，被屏東地院依詐欺罪，判處有期徒刑5月，案經上訴高雄高分院後，宣告緩刑3年定讞，緩刑期間交付保護管束，並賠償被害人共15萬元。',
  array([-8.8697138e+00, -1.4053787e-04], dtype=float32),
  1,
  'aml')]

批次測試

In [ ]:
data['raw_content'].str.replace('\（.*?\）', '')

In [27]:
pred_sentences = list(test['content'])
test_start_time = datetime.now()
predictions = getPrediction(pred_sentences)
print("Inference took time ", datetime.now() - test_start_time)

INFO:tensorflow:Writing example 0 of 10


INFO:tensorflow:Writing example 0 of 10


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


Inference took time  0:00:03.498000


In [28]:
for i in range(10):
    print(predictions[i])

('將近九十四萬的同意票，讓韓國瑜成為台灣自治史上首位被罷免的市長，從風光的立委到退隱政壇、從賣菜郎到藍營救世主、創下韓流奇蹟，短短一年半三次選舉，把他推向政治生命高峰也一舉跌落谷底，韓國瑜的政治人生暴起又暴落，轟轟烈烈，寫下台灣政壇傳奇。挽袖的藍襯衫、接地氣的庶民語言與肢體動作創造了「韓流」，打破綠營在港都執政20年的高牆，翻轉藍綠執政版圖，成為政壇耀眼的明星。而面臨總統大選後罷韓的明流暗湧，失去的民心有如慧星般急速隕落，失去了高雄市長的韓國瑜，政壇之路是否還有序曲？', array([-3.5404533e-05, -1.0249677e+01], dtype=float32), 0, 'non_aml')
('在非裔美人佛洛伊德遭警方不當暴力致死後，美國福斯新聞的一個節目5日用圖表指美國非裔人士遭受暴力時，股市竟會大漲，此舉引發嘩然。福斯新聞對此事發表聲明，對未顧慮他人感受致歉。《法新社》7日報導，福斯新聞的節目「拜爾特別報導」播出一張圖表，顯示美國非裔人士受到暴力，引發社會關注後，股市呈現走高情形。圖表中顯示，在佛洛伊德喪命後，標準普爾500指數大漲3.4%、民權領袖馬丁・路德・金恩遇刺後，標普500上漲2.9%、涉毆打非裔青年金恩的警察被無罪釋放，以及非裔少年布朗遭警方擊斃後，標普500的漲幅都是1.2%。伊利諾伊州聯邦眾議員拉什針對福斯新聞的行為甚為不滿，他痛批 :「這張圖表清楚地表明，福斯新聞並不在乎黑人的性命。」共和黨全國委員會前主席斯蒂爾也在推特上發文 :「這就是福斯新聞悼念黑人死亡的方式——用股市上漲多少來呈現。」福斯新聞6日晚間發表聲明表示，引發爭議的圖表原本是用來說明市場對歷史性時刻如何反應，不應該在沒有完整脈絡的情況下在電視上播出。福斯新聞說 :「我們對這張圖表未顧及他人感受致歉，並認真看待此事。」', array([-4.577532e-05, -9.992626e+00], dtype=float32), 0, 'non_aml')
('高雄市議長許崑源昨晚近9時，從住家17樓墜樓身亡，由於墜樓原因不明，引起各界議論紛紛。家屬今天發表3點聲明，呼籲各界不要以訛傳訛，尊重往生者為大，家屬對不實言論將保留法律追訴權。市議員宋立彬代表許崑源家屬，發布三點聲明，首先家屬對外界的關切表達感謝，但針對部分媒體不實報導與網路傳言表示譴責，強調均為子虛

In [127]:
predictions = getPrediction(val['raw_content'])

INFO:tensorflow:Writing example 0 of 2428


INFO:tensorflow:Writing example 0 of 2428


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: news_id


...

INFO:tensorflow:Restoring parameters from C:\Users\Jasonhsu\Desktop\esun\bert_classfication2\model.ckpt-1516


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


In [130]:
pd.DataFrame(predictions, columns = ['content','vct','pred','pred_disc']).to_excel('pred_bert.xlsx', index = False)

In [90]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
ws = WS("./data", disable_cuda=False)
pos = POS("./data", disable_cuda=False)
ner = NER("./data", disable_cuda=False)

In [92]:
del ws
del pos
del ner